# 早期終了 (Early stopping)

In [ ]:
# Tensorflowが使うCPUの数を制限します。(VMを使う場合)
%env OMP_NUM_THREADS=1
%env TF_NUM_INTEROP_THREADS=1
%env TF_NUM_INTRAOP_THREADS=1

from tensorflow.config import threading
num_threads = 1
threading.set_inter_op_parallelism_threads(num_threads)
threading.set_intra_op_parallelism_threads(num_threads)

#ライブラリのインポート
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import utils

## 過学習(Overtraining)の例
### データ点の生成
sin関数に0.3の大きさのガウシアンノイズがのったデータを考えます。データ数は30にします。

In [ ]:
# データ点の取得
x, t = utils.dataset_overtraining(n=30, noise_scale=0.3)

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots()

# データ点をプロット
ax.scatter(x, t, s=10, c="black", label="data")  # データ点のプロット
x_grid = np.linspace(-np.pi, np.pi, 100)
ax.plot(x_grid, np.sin(x_grid), c="blue", label="y=sin(x)")
ax.legend()

# 図を表示
plt.show()

### デモに用いる深層学習モデル
過学習の様子を見るために、パラメータ数の多いモデルを使ってフィットをしてみます。
ここでは、ノード数が128の隠れ層を5層重ねた多層パーセプトロンを使用します。
活性化関数としてはReLUを使い、モデルの出力の直前のノードは、活性化関数を使用しないことにします。
誤差関数は二乗和誤差を使い、最適化関数としてadamを使用します。

In [ ]:
# モデルの定義
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense

model = Sequential(
    [
        Input(shape=(1,)),
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dense(1, activation="linear"),  # ノード数が1の層を追加。活性化関数は線形関数。
    ]
)

# 誤差関数として二乗和誤差、最適化関数としてadamを使用
model.compile(loss="mean_squared_error", optimizer="adam")

# トレーニング
model.fit(x=x, y=t, epochs=1000, verbose=0)

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots()

# データ点をプロット
ax.scatter(x, t, s=10, c="black", label="data")  # データ点のプロット
ax.plot(x_grid, model(x_grid), c="red", label="prediction")
ax.plot(x_grid, np.sin(x_grid), c="blue", label="y=sin(x)")
ax.legend()

# 図を表示
plt.show()

モデルの出力(赤線)がトレーニングデータ点に強くフィットしてしまっていることがわかります。

## データの分割
Early stoppingでは データセットをランダムに2つに分割します。
一方のデータセット(training dataset)でトレーニングをし、もう一方のデータセット(validation dataset)で過学習がおこらないかを確認します。
サンプルの分割は手で行うこともできますが、scikit learnというライブラリに便利な関数が用意されているので、これを用いることにします。

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_valid, t_train, t_valid = train_test_split(x, t, test_size=0.30, random_state=0)

上の例では、全体の70%をトレーニングデータ、全体の30%を検証用データ(validation sample)として分割しています。

トレーニングデータ、検証用データをプロットすると下図のようになります。
黒点がトレーニングデータ、オレンジ点が検証用データです。
全データセットからランダムに分割されていることがわかります。

In [ ]:
# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots()

# データ点をプロット
ax.scatter(x_train, t_train, s=10, c="black", label="training data")
ax.scatter(x_valid, t_valid, s=10, c="orange", label="validation data")
ax.plot(x_grid, np.sin(x_grid), c="blue", label="y=sin(x)")
ax.legend()

# 図を表示
plt.show()

検証用データに対してロスの値を監視をするには、`fit`関数の中で、`validation_data`引数に検証用データを与えます。

In [ ]:
# モデルの定義
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense

model = Sequential(
    [
        Input(shape=(1,)),
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dense(1, activation="linear"),  # ノード数が1の層を追加。活性化関数は線形関数。
    ]
)

model.compile(loss="mean_squared_error", optimizer="adam")

# トレーニング
history = model.fit(x=x_train, y=t_train, validation_data=(x_valid, t_valid), epochs=300, verbose=0)

# ロス関数の推移を取得します
loss_train = history.history["loss"]
loss_valid = history.history["val_loss"]

結果をプロットしてみます。

In [ ]:
# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots(2, 1, figsize=(6, 9))

# データ点をプロット
ax[0].scatter(x_train, t_train, s=10, c="black", label="training data")
ax[0].scatter(x_valid, t_valid, s=10, c="orange", label="validation data")
ax[0].plot(x_grid, model(x_grid), c="red", label="prediction")
ax[0].plot(x_grid, np.sin(x_grid), c="blue", label="y=sin(x)")
ax[0].legend()

# ロス関数の推移をプロット
ax[1].plot(loss_train, label="loss (train)")
ax[1].plot(loss_valid, label="loss (valid)")
ax[1].set_xlabel("epochs")
ax[1].set_ylabel("loss")
ax[1].legend()

# 図を表示
plt.show()

学習に使った黒点に沿うようにモデルがフィットされていることがわかります。
一方で、検証用の点(オレンジ色の点)に対しての予測値は大きくずれてしまっています。

誤差関数の値(loos)の推移を見ると、epoch数が およそ80 あたりで、検証用データの誤差関数の値が下がらなくなってしまってしまい、学習を進めるにつれ、検証用データに対する誤差はむしろ大きくなってしまっています。
これが過学習を起こした状態で、トレーニングデータに強く学習してしまったため、本来の目的である汎化誤差が大きくなってしまっています。

これは、検証用データの誤差関数値がそれ以上下がらなくなった点で学習を止めることで緩和することができます。
上の例だとおよそ80 エポックぐらいで誤差関数値が上昇に転じているので、このあたりで学習を止めることにしましょう。

In [ ]:
# モデルの定義
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense

model = Sequential(
    [
        Input(shape=(1,)),
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dense(1, activation="linear"),  # ノード数が1の層を追加。活性化関数は線形関数。
    ]
)

model.compile(loss="mean_squared_error", optimizer="adam")

# トレーニング
history = model.fit(x=x_train, y=t_train, validation_data=(x_valid, t_valid), epochs=80, verbose=0)

# ロス関数の推移を取得します
loss_train = history.history["loss"]
loss_valid = history.history["val_loss"]

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots(2, 1, figsize=(6, 9))

# データ点をプロット
ax[0].scatter(x_train, t_train, s=10, c="black", label="training data")
ax[0].scatter(x_valid, t_valid, s=10, c="orange", label="validation data")
ax[0].plot(x_grid, model(x_grid), c="red", label="prediction")
ax[0].plot(x_grid, np.sin(x_grid), c="blue", label="y=sin(x)")
ax[0].legend()

# ロス関数の推移をプロット
ax[1].plot(loss_train, label="loss (train)")
ax[1].plot(loss_valid, label="loss (valid)")
ax[1].set_xlabel("epochs")
ax[1].set_ylabel("loss")
ax[1].legend()

# 図を表示
plt.show()

今度はどうなったでしょうか？おそらく、過学習が緩和されたのではないかと思います。

このように、学習を途中で打ち切るような手法を早期終了(Early Stopping)と言います。
Keras/Tensorflowでは、これが簡単に行えるようにコールバックが用意されています。

https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping

(他にもいろいろなコールバックが用意されています。コールバックは、学習の前後や学習の途中で追加で何か処理をしたい時に使います。)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=20, restore_best_weights=True)

`patience`というのは、モニターしたい値(ここでは検証用データのロス)が何エポック下がらなくなったら学習を止めるか、を指定します。今はちょっと大きめの値の20としておきます。
この値は問題・モデルによって調節するべき値です。小さすぎると、過学習を起こす前・学習が十分に行われていない段階で学習がストップしてしまいます。一方で、大きすぎると学習時間が余計にかかってしまいます。

`restore_best_weights`がFalseだと、early stoppingが終わったときのモデルの重みがそのまま残ります。そのため、`restore_best_weights`がFalseかつ`patience`を大きくすると、過学習が起こってしまった値が記録されてしまいます。この値をTrueにしておくと、ロスが最小になったときのモデルの重みでモデルがアップデートされます。

このコールバックを`fit` の引数として与えることで、Early stoppingが行われます。

In [ ]:
# モデルの定義
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential(
    [
        Input(shape=(1,)),
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dense(1, activation="linear"),  # ノード数が1の層を追加。活性化関数は線形関数。
    ]
)

model.compile(loss="mean_squared_error", optimizer="adam")

# 早期終了(Early stopping)コールバック
es = EarlyStopping(patience=20, restore_best_weights=True)

# トレーニング
history = model.fit(
    x=x_train,
    y=t_train,
    validation_data=(x_valid, t_valid),
    epochs=500,
    callbacks=es,  # ここでコールバックを指定します。
    verbose=0,
)

print(
    f"stopped epoch = {es.stopped_epoch}, patience = {es.patience}, best epoch = {es.stopped_epoch - es.patience}"
)

# ロス関数の推移を取得します
loss_train = history.history["loss"]
loss_valid = history.history["val_loss"]

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots(2, 1, figsize=(6, 9))

# データ点をプロット
ax[0].scatter(x_train, t_train, s=10, c="black", label="training data")
ax[0].scatter(x_valid, t_valid, s=10, c="orange", label="validation data")
ax[0].plot(x_grid, model(x_grid), c="red", label="prediction")
ax[0].plot(x_grid, np.sin(x_grid), c="blue", label="y=sin(x)")
ax[0].legend()

# ロス関数の推移をプロット
ax[1].plot(loss_train, label="loss (train)")
ax[1].plot(loss_valid, label="loss (valid)")
ax[1].set_xlabel("epochs")
ax[1].set_ylabel("loss")
ax[1].legend()

# 図を表示
plt.show()

指定した最大エポック数(500エポック)の前に学習が終了していると思います。また、過学習も緩和されています。